In [ ]:
import pandas as pd
import numpy as np

import torch
from torch import nn

from sklearn.model_selection import train_test_split

import shap

### 1.   Prepare input data
-----

In [ ]:
# Function: determine PHA-L read cut-offs for binary classification 
def categorize_lectin(data_all, quantile_high, quantile_low, ref_col_loc):
    cutoff = np.quantile(data_all.iloc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
    print(f"Cut-off for PHA-L high: {cutoff[0]}; Cut-off for PHA-L low: {cutoff[1]}")
    
    high_indices = np.array(data_all.iloc[:,ref_col_loc]>=cutoff[0])
    low_indices = np.array(data_all.iloc[:,ref_col_loc]<cutoff[1])
    high_low_indices = np.logical_or(high_indices, low_indices)

    high_count = high_indices.sum()
    low_count = low_indices.sum()
    
    return cutoff, [high_indices, low_indices, high_low_indices], [high_count, low_count]

In [ ]:
# Load input file
input_df = pd.read_csv('/Users/mahallab/Desktop/Single cell data/TIL scRNA-seq/scData_TIL_Jan9_2022/TIL_ProjectTIL_data_final.csv')

In [ ]:
# Process data: binary classification
quantile_high, quantile_low = 0.75, 0.25
cutoff, indices, count = categorize_lectin(input_df, quantile_high, quantile_low, -1)

input_df.loc[indices[0], "PHA-L"] = 1
input_df.loc[indices[1], "PHA-L"] = 0

input_df = input_df.loc[indices[2], :]

In [ ]:
#y: class array
y = input_df['PHA-L'].values 
#X: transcript data array
X = input_df.iloc[:, 1:-1].values

In [ ]:
# Split training, validation and test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=342, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=2, stratify=y_train_val)

### 2.   Load model
-----

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.4),
            nn.BatchNorm1d(64),
            nn.Linear(64, 16),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(16),
            nn.Linear(16, 8),
            nn.LeakyReLU(),
            nn.BatchNorm1d(8),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        y = self.linear_relu_stack(x)
        return y

In [ ]:
model_eval = torch.load('Model.pth')

### 3.   SHAP analysis
-----

In [ ]:
# Define SHAP analysis function
def shap_explainer(model, data, feature_list):
    explainer = shap.DeepExplainer(model, data)
    shap_values = explainer.shap_values(data)

    # summary plot
    shap.summary_plot(shap_values, features=data, feature_names=feature_list, max_display=25, show=True, plot_size=(10,15))
    
    # save basic stats of shap values
    shap_values_df = pd.DataFrame(shap_values)
    shap_values_summary = pd.DataFrame(np.abs(shap_values_df).mean(), columns=['Mean Absolute SHAP value'])
    shap_values_summary['Max Absolute SHAP value'] = np.abs(shap_values_df).max()
    shap_values_summary['Median Absolute SHAP value'] = np.abs(shap_values_df).median()
    shap_values_summary["Gene"] = feature_list

    return explainer, shap_values, shap_values_summary

In [ ]:
# SHAP analysis
gene_list = input_df.columns[1:-1]

rng = np.random.default_rng(1111)
random_sample_num = 1000
random_index = rng.choice(range(X_train.shape[0]), random_sample_num, replace = False, shuffle = False)
random_train_data = torch.tensor(X[random_index]).float()

explainer_training, shap_values_training, shap_values_summary = shap_explainer(model_eval, random_train_data, gene_list)